In [1]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
from grouper import group_by_volume, group_by_time
from pyfolio.timeseries import perf_stats
from utils import chart_price, plot, perf, v_backtester, c_backtester, perf_var, summary
import sys
sys.path.append('/home/tomek/ib_tools')
import matplotlib.pyplot as plt
%matplotlib inline
from tester import get_data
from numba import jit

In [2]:
df = get_data('NQ')
#df = df.loc['20190915':]

In [3]:
dfp = df[[ 'high', 'low',]]
n = dfp.to_numpy()
n.shape

(464910, 2)

In [4]:
@jit(nopython=True)
def bar(data: np.array, box: float, threshold: int = 3) -> np.array:
    filter = box * threshold
    extreme = data[0, 0]
    state = np.ones((data.shape[0], 1))
    count = np.ones(state.shape)
    for i, row in enumerate(data):      
        if i == 0:
            continue
        box_test =  ((row[0] - extreme) >= box, (extreme - row[1]) >= box)
        swing_test = ((extreme - row[1]) >= filter, (row[0] - extreme) >= filter)
        #print(f'extreme: {extreme}')
        #print(f'row: {row}')
        #print(f'box_test: {box_test}, swing_test: {swing_test}')
        #print(f'box: {((row[1] - extreme), (extreme - row[0]))}')
        #print(f'swing {((extreme - row[1]), (row[0] - extreme))}')
        
        if state[i-1, 0] == 1:
            if box_test[0]:
                state[i] = 1
                extreme = row[0]
                count[i] = count[i-1] + 1
            else:
                if swing_test[0]:
                    state[i] = -1
                    extreme = row[1]
                    count[i] = 1
                else:
                    state[i] = 1
                    count[i] = count[i-1]
                    
        elif state[i-1, 0] == -1:
            if box_test[1]:
                state[i] = -1
                extreme = row[1]
                count[i] = count[i-1] + 1
            else:
                if swing_test[1]:
                    state[i] = 1
                    extreme = row[0]
                    count[i] = 1
                else:
                    state[i] = -1
                    count[i] = count[i-1]
        
        else: 
            raise ValueError('Wrong state value!')
        #print(f'state: {state[i]}, count: {count[i]}')
        #print('----------------------')
            
    return np.concatenate((state, count), axis=1)

In [5]:
%%timeit
bar(n, 50)

14 ms ± 148 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [6]:
df[['point', 'box']] = bar(n, 30, 3)
df['box'] = df['box'] * df['point']
df

,open,high,low,close,volume,average,barCount,point,box
date,,,,,,,,,
2018-09-03 00:00:00,7670.00,7675.00,7670.00,7671.50,166,7672.550,104,1.0,1.0
2018-09-03 00:01:00,7671.50,7672.75,7670.00,7670.00,84,7671.675,60,1.0,1.0
2018-09-03 00:02:00,7670.00,7671.00,7669.75,7671.00,28,7670.250,21,1.0,1.0
2018-09-03 00:03:00,7671.25,7672.25,7671.25,7671.75,45,7671.850,31,1.0,1.0
2018-09-03 00:04:00,7672.00,7672.00,7669.75,7669.75,32,7670.775,26,1.0,1.0
...,...,...,...,...,...,...,...,...,...
2019-12-31 22:55:00,8769.75,8770.75,8768.75,8770.00,83,8769.825,50,-1.0,-2.0
2019-12-31 22:56:00,8770.25,8771.25,8770.25,8770.25,41,8770.650,19,-1.0,-2.0
2019-12-31 22:57:00,8770.50,8771.00,8769.75,8769.75,48,8770.250,25,-1.0,-2.0


In [7]:
summary(df['close'], df['box'], slip=1.5).stats

,0,1,3,5,6,10,15,17,19
Annual return,-0.0357971,-0.0629924,0.0417306,0.0465992,0.00612029,0.00762587,-0.01266,-0.00545367,-0.00854589
Cumulative returns,-0.0484064,-0.084751,0.0572242,0.0639552,0.00833959,0.0103939,-0.0171922,-0.00741573,-0.0116139
Annual volatility,0.188529,0.159454,0.115201,0.086724,0.0771129,0.0384743,0.0132398,0.00635634,0.0099548
Sharpe ratio,-0.0992646,-0.328709,0.412009,0.568114,0.117224,0.216524,-0.955647,-0.857143,-0.857143
Calmar ratio,-0.208631,-0.407593,0.358378,0.649185,0.0697863,0.251562,-0.582773,-0.735419,-0.735831
Stability,0.00167359,0.0347691,0.379073,0.682447,0.155868,0.225887,0.668407,0.63724,0.63724
Max drawdown,-0.171581,-0.154547,-0.116443,-0.0717812,-0.0877005,-0.0303141,-0.0217237,-0.00741573,-0.0116139
Omega ratio,0.982599,0.942763,1.0887,1.16332,1.03632,1.13919,0.211839,0,0
Sortino ratio,-0.140923,-0.483505,0.678675,0.983256,0.201872,0.368745,-0.985798,-0.857143,-0.857143
Skew,-0.0622279,0.471105,1.23124,2.27263,2.87671,3.78367,-11.5307,-18.4392,-18.4392
